In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import shutil
import cv2
import imutils
import seaborn as sns
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model, Sequential
import keras
from PIL import Image

import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, matthews_corrcoef

In [ ]:
%cd /content/drive/My Drive/

In [ ]:
# prompt: set seed to 42 for keras and all stuff in code
import tensorflow as tf
import random as rn

seed = 42

os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)

In [ ]:
#Define a function that crop tha brain contour
def crop_brain_contour(image, plot=False):

    #Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    #Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    #Extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    #Crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]

    if plot:
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)
        plt.tick_params(axis='both', which='both',top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        plt.title('Cropped Image')
        plt.show()

    return new_image

In [ ]:
#Define a function that load data
def load_data(dir_list, image_size):

    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size

    for directory in dir_list:
        for filename in os.listdir(directory):
            image = cv2.imread(directory+'/'+filename)
            image = crop_brain_contour(image, plot=False)
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = image / 255.
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

In [ ]:
#Load trainig data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_train, y_train = load_data(['Dataset/train/yes', 'Dataset/train/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
#Load test data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_test, y_test = load_data(['Dataset/test/yes', 'Dataset/test/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
#Load validation data
IMG_WIDTH, IMG_HEIGHT = (240, 240)
X_val, y_val = load_data(['Dataset/val/yes', 'Dataset/val/no'], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from keras.layers import Dense, Activation, Dropout


base_Neural_Net= ResNet101V2(input_shape=(240,240,3), weights='imagenet', include_top=False)
model=Sequential()
model.add(base_Neural_Net)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

for layer in base_Neural_Net.layers:
    layer.trainable = False


model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

# Set up ModelCheckpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('Models/best4_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model with checkpoint
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_val, y_val), callbacks=[checkpoint])

# Load the best model
model.load_weights('Models/best4_model.h5')

In [ ]:
h = history.history
plt.plot(h['accuracy'], label = 'accuracy')
plt.plot(h['val_accuracy'], label = 'val-accuracy')
plt.title('Accuracy vs Val Accuracy')
plt.legend()
plt.show()

In [ ]:
#Plot the graphical interpretation
h = history.history
plt.plot(h['loss'], label = 'loss')
plt.plot(h['val_loss'], label = 'val-loss')
plt.title('Loss vs Val Loss')
plt.legend()
plt.show()

In [ ]:
#Test our model on the test set
model = load_model('Models/best4_model.h5')
acc = model.evaluate(X_test, y_test)[1]
print(f'The accuracy of our model is {acc}')

In [ ]:
model = load_model('Models/best4_model.h5')
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Convert predicted values to binary values using threshold of 0.5
y_test_pred = np.where(y_test_pred >= 0.5, 1, 0)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

# Plot confusion matrix as heatmap
df_cm = pd.DataFrame(cm, index=[i for i in ["1",'0']], columns=[i for i in ["1","0"]])
plt.figure(figsize=(4,3))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
acc= (tp+tn)/ (tp+tn+fp+fn)
pre= tp/(fp+tp)
rec=tp/(fn+tp)
f1 = 2*pre*rec/(pre+rec)
specificity = tn / (tn + fp)
fpr = fp / (tn + fp)
fnr = fn / (fn + tp)
npv = tn / (tn + fn)
fdr = fp / (fp + tp)
mcc = matthews_corrcoef(y_test, y_test_pred)
print('ResNet101 V2 Model additional performance metrics for Test set')
print('- Accuracy: {:.4f}'.format(acc))
print('- Precision: {:.4f}'.format(pre))
print('- Recall: {:.4f}'.format(rec))
print('- Specificity: {:.4f}'.format(specificity))
print('- F1 score: {:.4f}'.format(f1))
print('- FPR: {:.4f}'.format(fpr))
print('- FNR: {:.4f}'.format(fnr))
print('- NPV: {:.4f}'.format(npv))
print('- FDR: {:.4f}'.format(fdr))
print('- MCC: {:.4f}'.format(mcc))